In [1]:
import sys
sys.path.append( "/notebook/nas-trainings/arne/DGFISMA_paragraph_detection"  ) 

In [2]:
from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

with open( "/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/typesystems/typesystem.xml"   , 'rb') as f:
    typesystem = load_typesystem(f)
    
f = open('/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/xmi/double_nested_list.xmi' ).read()
#then also test on the previous one

#f = open('/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/xmi/small_nested_tables.xmi' ).read()

#f = open('/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/xmi/minus_lesser_of.xmi' ).read()

cas=load_cas_from_xmi(f , typesystem=typesystem  )

In [3]:
from src.annotations import annotate_lists_eurlex_html
cas.get_view( 'html2textView' ).select(  "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" )

annotate_lists_eurlex_html( cas, typesystem, 'html2textView'  ) 

In [4]:
paragraphs=cas.get_view( 'html2textView' ).select( 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph')
list( paragraphs )

[de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9540, begin=2292, end=11397, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9536, begin=2710, end=3915, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9539, begin=4058, end=11363, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9537, begin=7653, end=8916, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9538, begin=9705, end=10946, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'

In [3]:
#do post request to paragraph annotation app
import sys
from typing import List, Tuple, Set, Generator

from cassis import Cas, TypeSystem

def annotate_lists_eurlex_html( cas: Cas, typesystem: TypeSystem, SofaID: str , value_between_tagtype_generator: Generator, end=-1 ) -> Cas:

    '''
    Given a cas and its typesystem, this function annotates list and sublists. Returns the same cas object as the input cas, but now with annotations added.
    '''
    
    Paragraph=typesystem.get_type( 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph' )

    cas_view=cas.get_view( SofaID )
    
    paragraph_list=[]
    detected_p=None

    for tag in value_between_tagtype_generator:
        
        #first check if not exceed the span of the covering table (for nested lists), if so rewind and stop.
        if end>0 and tag.begin > end:
            
            #Add the last detected list to the cas
            if len( paragraph_list )>1:
                cas_view.add_annotation( Paragraph( begin=paragraph_list[0].begin, end=paragraph_list[-1].end , divType="enumeration" ) )     
            
            value_between_tagtype_generator.rewind()

            return
        
        if tag.tagName=="p":
            
            detected_p=tag
            continue
        
        if tag.tagName=="table":
            
            #if no p has been detected before, ignore this detected table
            if not detected_p:
                continue
                        
            tags_covered=list(cas_view.select_covered( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType", tag ))
            if contains_table( tags_covered[1:] ):
                #print( "table containing nested_list_detected: range", tag , tag.begin, tag.end )
                annotate_lists_eurlex_html(cas, typesystem, SofaID, value_between_tagtype_generator, end=tag.end  )
                
            #2) check if it is not one of the nested tags (but don't check this for the very first detected table, when paragraph list is still empty):            
            #if paragraph_list:
            #    if tag.begin < paragraph_list[-1].end:
            #        continue              
            
            #3) start of a new list if previous tag was a <p> (structure of a list is always <p></p> <table></table> <table></table>)
            #So if there is no text between the end of the detected p and the table_tag ==> start of a new list
            if not cas_view.sofa_string[ detected_p.end:tag.begin ].strip():
                
                #so it is indeed the start of a new list ==> add the old list as annotation to the cas
                if len(paragraph_list)>1:
                    
                    #But detected p that is located just before table tag, should also be reasonably long, to initiate new list. 
                    #E.g. this: <p> start of new list </p> <table> </table> <table></table> <p> minus the lesser of: </p> <table> </table>. 
                    #Second p should not initiate new list
                    if len( cas_view.sofa_string[ paragraph_list[-1].end:tag.begin ].split())<=5:
                        paragraph_list.append( detected_p )
                        paragraph_list.append( tag )
                        continue
                    cas_view.add_annotation( Paragraph( begin=paragraph_list[0].begin, end=paragraph_list[-1].end , divType="enumeration" ) )
                paragraph_list=[]                
                paragraph_list.append( detected_p ) #the p
                paragraph_list.append( tag )  #the table tag

            elif paragraph_list:
                 #there should not be any text between the beginning of the current and the end of the previous table tag
                if not(cas_view.sofa_string[ paragraph_list[-1].end:tag.begin ].strip() ):        
                    paragraph_list.append( tag )
        
    #add the last detected list to the cas
    if len( paragraph_list )>1:
        cas_view.add_annotation( Paragraph( begin=paragraph_list[0].begin, end=paragraph_list[-1].end , divType="enumeration" ) )            
      
    return cas

            
def contains_table(  list_of_value_between_tagtype ):
    for tag in list_of_value_between_tagtype:
        if tag.tagName=='table':
            return True
    return False
                


In [4]:
from src.annotations import annotate_lists_eurlex_html
cas.get_view( 'html2textView' ).select(  "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" )

annotate_lists_eurlex_html( cas, typesystem, SofaID, ) 


In [4]:
vbtt_iterator=cas.get_view( 'html2textView' ).select(  "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" )
seek_vbtt=seekable_iterator( vbtt_iterator )

cas=annotate_lists_eurlex_html( cas , typesystem , 'html2textView' , seek_vbtt   )

In [5]:
paragraphs=cas.get_view( 'html2textView' ).select( 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph')
list( paragraphs )

[de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9540, begin=2292, end=11397, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9536, begin=2710, end=3915, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9539, begin=4058, end=11363, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9537, begin=7653, end=8916, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9538, begin=9705, end=10946, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'

In [9]:
with open(  "/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/xmi/double_nested_list_lists.txt" , "w" ) as f:
    for par in cas.get_view( 'html2textView' ).select( 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'):
        f.write( f"{par.get_covered_text()}\n"  )
        f.write( "_"*100 )
        f.write( "\n" )


Article 3
Determination of total amount of annual contributions to be raised
The total amount of annual contributions to be raised for a given financial year shall be calculated on the basis of Part I of the budget adopted by the Board for that financial year in accordance with Article 61 of Regulation (EU) No 806/2014, adjusted by the result of the budget of the latest financial year for which the final accounts have been published in accordance with Article 63(7) of that Regulation.
The Board shall determine the total amount of annual contributions so that Part I of the budget of the Board is balanced.
Article 4
Allocation of total amount of annual contributions
1.   The total amount to be raised in accordance with Article 3 shall be allocated as follows:
(a)
95 % to the following entities and groups:
(i)
entities and groups referred to in Article 7(2) of Regulation (EU) No 806/2014;
(ii)
entities and groups with regard to which the Board has decided to exercise powers under Regulati

In [ ]:
import json
file=dict()

file[ 'text' ]=open( "../tests/test_files/html/double_nested_list.html"  ).read()

with open(  "../tests/test_files/html/double_nested_list.json", "w"  ) as f:
    f.write( json.dumps(file, indent = 4)  )

In [2]:
#curl -H "Content-Type: application/json" -d @double_nested_list.json -X POST https://uima.dev.dgfisma.crosslang.com/html2text > double_nested_list.xmi

In [3]:
output_json={}

f = open('/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/xmi/double_nested_list.xmi' ).read()

cas=load_cas_from_xmi(f , typesystem=typesystem  )
 
output_json[ 'cas_content' ] = base64.b64encode(bytes( cas.to_xmi() , 'utf-8')).decode()
output_json[ 'content_type' ] = 'html'

#import json
#with open('/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/json/double_nested_list.json', 'w') as fp:
#    json.dump(output_json, fp)

NameError: name 'load_cas_from_xmi' is not defined

In [44]:
#do post request to paragraph annotation app
import sys
from typing import List, Tuple, Set

from cassis import Cas, TypeSystem

def annotate_lists_eurlex_html( cas: Cas, typesystem: TypeSystem, SofaID: str , list_of_value_between_tagtype: list ) -> Cas:

    '''
    Given a cas and its typesystem, this function annotates list and sublists. Returns the same cas object as the input cas, but now with annotations added.
    '''
    
    Paragraph=typesystem.get_type( 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph' )

    cas_view=cas.get_view( SofaID )

    paragraph_list=[]
    detected_p=None
    
    for i, tag in enumerate(list_of_value_between_tagtype):
            
        if tag.tagName =='p':
            
            #keep track of detected p's
            detected_p = tag
            continue
            
        if tag.tagName == 'table': 
            
            #if no p has been detected before, ignore this detected table
            if not detected_p:
                continue
            
            #1) check if it contains a nested table, if so, recursive function call:
            tags_covered=list(cas_view.select_covered( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType", tag ))
            if contains_table( tags_covered[1:] ): #this means it is a nested structure ( the first element of tags_covered is tag )
                annotate_lists_eurlex_html( cas, typesystem, SofaID, tags_covered[1:] )
            
            #2) check if it is not one of the nested tags (but don't check this for the very first detected table, when paragraph list is still empty):            
            if paragraph_list:
                if tag.begin < paragraph_list[-1].end:
                    continue              
            
            #3) start of a new list if previous tag was a <p> (structure of a list is always <p></p> <table></table> <table></table>)
            #So if there is no text between the end of the detected p and the table_tag ==> start of a new list
            if not cas_view.sofa_string[ detected_p.end:tag.begin ].strip():
                
                #but first check if the detected p is not part of the previous detected table. 
                #i.e. p's like this: <table><p></p></table><table></table>, should not start a new list
                #If so ==> no new list is detected, and table tag is just added to paragraph_list
                if paragraph_list:
                    if (detected_p.begin < paragraph_list[-1].end) and not(cas_view.sofa_string[ paragraph_list[-1].end:tag.begin ].strip() ): 
                        paragraph_list.append( tag )
                        continue
                
                #so it is indeed the start of a new list ==> add the old list as annotation to the cas
                if len(paragraph_list)>1:
                    
                    #But detected p that is located just before table tag, should also be reasonably long, to initiate new list. 
                    #E.g. this: <p> start of new list </p> <table> </table> <table></table> <p> minus the lesser of: </p> <table> </table>. 
                    #Second p should not initiate new list
                    if len( cas_view.sofa_string[ paragraph_list[-1].end:tag.begin ].split())<=5:
                        paragraph_list.append( detected_p )
                        paragraph_list.append( tag )
                        continue
                    cas_view.add_annotation( Paragraph( begin=paragraph_list[0].begin, end=paragraph_list[-1].end , divType="enumeration" ) )
                paragraph_list=[]                
                paragraph_list.append( detected_p ) #the p
                paragraph_list.append( tag )  #the table tag

            elif paragraph_list:
                 #there should not be any text between the beginning of the current and the end of the previous table tag
                if not(cas_view.sofa_string[ paragraph_list[-1].end:tag.begin ].strip() ):        
                    paragraph_list.append( tag )
        
    #add the last detected list to the cas
    if len( paragraph_list )>1:
        cas_view.add_annotation( Paragraph( begin=paragraph_list[0].begin, end=paragraph_list[-1].end , divType="enumeration" ) )
        
    return cas

#helper function

def contains_table(  list_of_value_between_tagtype ):
    for tag in list_of_value_between_tagtype:
        if tag.tagName=='table':
            return True
    return False

In [174]:
vbtt_iterator=cas.get_view( 'html2textView' ).select(  "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" )
seek=seekable_iterator( vbtt_iterator )

In [67]:
class seekable_iterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.current = None
        self.reuse = False

    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()

    def next(self):
        if self.reuse:
            self.reuse = False
        else:
            self.current = None
            self.current = next(self.iterator)
        return self.current

    def rewind(self):
        self.reuse = True

In [168]:
seek=seekable_iterator(  iter([1,2,3,4,5])  )

In [169]:
def iterate(  seek, end=1000 ):
    for item in seek:
        
        print( item )
        
        if item==2:
            print( "recursive" )
            iterate(  seek, end=4  )
        
        if item >= end:    
            seek.rewind()
            return "test"
            

In [170]:
iterate( seek )

1
2
recursive
3
4
4
5


In [148]:
seek.rewind()

In [149]:
seek.next()

2

In [115]:
class bidirectional_iterator:
    def __init__(self, collection):
        self.collection = collection
        self.index = 0

    def next(self):
        try:
            result = self.collection[self.index]
            self.index += 1
        except IndexError:
            raise StopIteration
        return result

    def prev(self):
        self.index -= 1
        if self.index < 0:
            raise StopIteration
        return self.collection[self.index]

    def __iter__(self):
        return self

In [137]:
bi_iter=bidirectional_iterator( [  1,2,3 ]    )

In [26]:
import json
import base64
import re

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

with open( "/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/typesystems/typesystem.xml"   , 'rb') as f:
    typesystem = load_typesystem(f)

with open('/notebook/nas-trainings/arne/DGFISMA_paragraph_detection/tests/test_files/response_json/double_nested_list_response.json') as json_file:
    response = json.load(json_file)
    
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem  )
    

In [37]:
list_of_paragraphs=cas.get_view( 'html2textView' ).select(  "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph" )
vbtts=cas.get_view( 'html2textView' ).select(  "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" )

In [38]:
list( list_of_paragraphs )

[de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9542, begin=2292, end=11397, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9536, begin=2710, end=3915, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9539, begin=4058, end=11363, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9537, begin=7653, end=8916, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph'),
 de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Paragraph(xmiID=9540, begin=7653, end=8916, divType='enumeration', id=None, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph')

In [35]:
for par in list_of_paragraphs:
    print( par.get_covered_text() )
    print( "_"*100 )

1.   Institutions shall report to the competent authorities, in accordance with the reporting requirements set out in Article 415(1) and the uniform reporting formats referred to in Article 415(3), the following items and their components in order to allow an assessment of the availability of stable funding:
   
      
      
      
         
            
               (a)
            
            
               the following own funds, after deductions have been applied, where appropriate:
               
                  
                  
                  
                     
                        
                           (i)
                        
                        
                           tier 1 capital instruments;
                        
                     
                  
               
               
                  
                  
                  
                     
                        
                           (ii)
            

In [29]:
for tag in vbtts:
    if tag.tagName == 'p':
        print( tag.get_covered_text() )  

5.   Competent authorities may waive the condition set out in point (d) of paragraph 4 where Article 20(1)(b) is applied. In that case additional objective criteria as set out in the delegated act referred to in Article 460 have to be met. Where such higher inflow is permitted to be applied, the competent authorities shall inform EBA about the result of the process referred to in Article 20(1)(b). Fulfilment of the conditions for such higher inflows shall be regularly reviewed by the competent authorities.
6.   EBA shall develop draft regulatory technical standards to further specify the additional objective criteria referred to in paragraph 5.
EBA shall submit those draft regulatory technical standards to the Commission by 1 January 2015.
Power is delegated to the Commission to adopt the regulatory technical standards referred to in the first subparagraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.
7.   Institutions shall not report inflows from any of the li